In [1]:
import uvicorn
from fastapi import FastAPI, Form, Request
from fastapi.templating import Jinja2Templates
from fastapi.responses import HTMLResponse
from src.MLProject.pipeline.prediction import PredictionPipeline
import pandas as pd
import os
from pathlib import Path
from fastapi.staticfiles import StaticFiles
from enum import Enum

app = FastAPI()

app.mount("/static", StaticFiles(directory="html_directory/static"), name="static")

templates = Jinja2Templates(directory='html_directory')

class Droppoint(str, Enum):
    Option1="Côte d Ivoire"
    Option2="Vietnam"
    Option3="Nigeria"
    Option4="Zambia"
    Option5="Tanzania"
    Option6="Rwanda"
    Option7="Haiti"
    Option8="Zimbabwe"
    Option9="Ethiopia"
    Option10="Others"
    Option11="South Africa"
    Option12="Guyana"
    Option13="Mozambique"
    Option14="Kenya"
    Option15="Uganda"
    Option16="South Sudan"
    Option17="Congo, DRC"

class ShipmentMode(str, Enum):
    Option1="Air"
    Option2="Truck"
    Option3="Air Charter"
    Option4="Ocean"

class DosageForm(str, Enum):
    Option1="Test kit"
    Option2="Others"
    Option3="Tablet"
    Option4="Tablet - FDC"

class DeliveryStatus(str, Enum):
    Option1="On time"
    Option2="Delayed"
    Option3="Delivered Early"

class PickupPoint(str, Enum):
    Option1="India"
    Option2="Others"


@app.get('/train')
async def training():
    os.system('python main.py')
    return "Training Successful"

@app.get("/home/", response_class=HTMLResponse)
async def hello(request: Request):
    return templates.TemplateResponse('index.html', {'request': request})

@app.post('/predict/', response_class=HTMLResponse)
async def prediction(request: Request,
    Drop_point: Droppoint = Form(...),
    Shipment_Mode: ShipmentMode = Form(...),
    Dosage_Form: DosageForm = Form(...),
    Line_Item_Quantity: str = Form(...),
    Pack_Price: str = Form(...),
    Unit_Price: str = Form(...),
    Weight: str = Form(...),
    Freight_Cost: str = Form(...),
    Line_Item_Insurance: str = Form(...),
    Delivery_Status: DeliveryStatus = Form(...),
    Pickup_Point: PickupPoint = Form(...)
):
    
    print(Drop_point, Shipment_Mode, Dosage_Form, Line_Item_Quantity, Pack_Price, 
          Unit_Price, Weight, Freight_Cost, Line_Item_Insurance, Delivery_Status, Pickup_Point)

    test_data = [Drop_point, Shipment_Mode, Dosage_Form, Line_Item_Quantity,
                 Pack_Price, Unit_Price, Weight, Freight_Cost,
                 Line_Item_Insurance, Delivery_Status, Pickup_Point]

    columns = ['Drop_point', 'Shipment_Mode', 'Dosage_Form', 'Line_Item_Quantity',
               'Pack_Price', 'Unit_Price', 'Weight', 'Freight_Cost',
               'Line_Item_Insurance', 'Delivery_Status', 'Pickup_Point']

    input_df = pd.DataFrame([test_data], columns=columns)
    pipeline = PredictionPipeline()
    pred = pipeline.predict(input_df)
    Results = round(pred[0], 2)
    return templates.TemplateResponse('results.html', {'request': request, 'result': Results})

if __name__ == "__main__":
    uvicorn.run(app, host='0.0.0.0', port=8000)

ModuleNotFoundError: No module named 'Shipment'